## CS-470: Homework Set # 2: Schedule for Loop pip
#### Name: GAO Xufeng. Sciper: 337088

In [1]:
import json
import numpy as np
import copy
import ctypes
import math  

In [2]:
class schedule_cell:
    
    def __init__(self, addr=None, cycle=None):
        self.addr = addr
        self.cycle = cycle
        self.ALU0 = 'nop'
        self.ALU1 = 'nop'
        self.Mult = 'nop'
        self.Mem = 'nop'
        self.Branch = 'nop'
    
    def set_slot(self, slot_type, ID):
        if slot_type == 'ALU0':
            self.ALU0 = ID
        if slot_type == 'ALU1':
            self.ALU1 = ID
        if slot_type == 'Mult':
            self.Mult = ID
        if slot_type == 'Mem':
            self.Mem = ID
        if slot_type == 'Branch':
            self.Branch = ID

    def set_reserved(self, slot_type, ID):
        if slot_type == 'ALU0':
            self.ALU0 = 'Reserved'
        if slot_type == 'ALU1':
            self.ALU1 = 'Reserved'
        if slot_type == 'ALU':
            if self.ALU0_empty():
                slot_type = 'ALU0'
                self.ALU0 = 'Reserved'
            elif self.ALU1_empty():
                slot_type = 'ALU1'
                self.ALU1 = 'Reserved'
        if slot_type == 'Mult':
            self.Mult = 'Reserved'
        if slot_type == 'Mem':
            self.Mem = 'Reserved'
        if slot_type == 'Branch':
            self.Branch = 'Reserved'
            
        return slot_type
    
    def append_consume_reg_slot(self, i, slot_type, consume_reg):
        if slot_type == 'ALU0':
            self.ALU0[i].append(consume_reg)
        if slot_type == 'ALU1':
            self.ALU1[i].append(consume_reg)
        if slot_type == 'Mult':
            self.Mult[i].append(consume_reg)
        if slot_type == 'Mem':
            self.Mem[i].append(consume_reg)
            
    def insert_clean_dep_slot(self, slot_type, clean_dep):
        if slot_type == 'ALU0':
            self.ALU0 = clean_dep
        if slot_type == 'ALU1':
            self.ALU1 = clean_dep
        if slot_type == 'Mult':
            self.Mult = clean_dep
        if slot_type == 'Mem':
            self.Mem = clean_dep     
        
            
    def insert_imm(self, i, slot_type, imm_ind, imm):
        if slot_type == 'ALU0':
            self.ALU0[i].insert(imm_ind, imm)
        if slot_type == 'ALU1':
            self.ALU1[i].insert(imm_ind, imm)
        if slot_type == 'Mult':
            self.Mult[i].insert(imm_ind, imm)
        if slot_type == 'Mem':
            self.Mem[i][imm_ind] = str(imm)+'('+self.Mem[i][imm_ind]+')'
            
    def return_ID(self, slot_type):
        ID = 'nop'
        if slot_type == 'ALU0':
            ID = self.ALU0
        if slot_type == 'ALU1':
            ID = self.ALU1 
        if slot_type == 'Mult':
            ID = self.Mult
        if slot_type == 'Mem':
            ID = self.Mem 
        if slot_type == 'Branch':
            ID = self.Branch
        return ID
        
    def set_addr(self, addr):
        self.addr = addr   
    def ALU0_empty(self):
        return self.ALU0 == 'nop'
    def ALU1_empty(self):
        return self.ALU1 == 'nop'
    def Mult_empty(self):
        return self.Mult == 'nop'
    def Mem_empty(self):
        return self.Mem == 'nop'
    def Branch_empty(self):
        return self.Branch == 'nop'
    
    def show_schedule_cell(self):
        print('addr: {}, ALU0: {}, ALU1: {}, Mult: {}, Mem: {}, Branch: {}, cycle: {}'.format(self.addr, self.ALU0, self.ALU1, self.Mult, self.Mem, self.Branch, self.cycle))
    
    def output_schedule_cell(self, slot_type):
        cell = 'nop'
        predicate = None
        if slot_type == 'ALU0' and not self.ALU0_empty():
            for ID, value in self.ALU0.items():
                predicate = ID
                cell = value
                cell = (cell[0]) + ' ' + ', '.join(char for char in cell[1:None])
        if slot_type == 'ALU1' and not self.ALU1_empty():
            for ID, value in self.ALU1.items():
                predicate = ID
                cell = value
                cell = (cell[0]) + ' ' + ', '.join(char for char in cell[1:None])
        if slot_type == 'Mult' and not self.Mult_empty():
            for ID, value in self.Mult.items():
                predicate = ID
                cell = value
                cell = (cell[0]) + ' ' + ', '.join(char for char in cell[1:None])
        if slot_type == 'Mem' and not self.Mem_empty():
            for ID, value in self.Mem.items():
                predicate = ID
                cell = value
                cell = (cell[0]) + ' ' + ', '.join(char for char in cell[1:None])
        if slot_type == 'Branch' and not self.Branch_empty():
            for ID, value in self.Branch.items():
                predicate = ID
                cell = value
                cell = (cell[0]) + ' ' + ', '.join(char for char in cell[1:None])
        return predicate, cell
    
        

In [3]:
class Compiler_state:
    def __init__(self):
        
        # input instruction sets
        self.Ins_set = []
        self.loop_region = None
        self.BB0 = None
        self.BB1 = None
        self.BB2 = None
        self.ID = 0
        self.chrID = [chr(i) for i in range(65,91)]
        self.Dependency_tab = {'Instr_Addr':[], 'ID':[], 'opcode':[], 'Des_reg':[], 'Local_dep':[],\
                               'InterLoop_dep':[], 'LoopInvar_dep':[], 'PostLoop_dep':[],\
                               'Consume_reg':[]}
        self.schedule_tab = []
        self.schedule_addr_tab = {}
        self.BB0_end = 0
        self.BB1_start = 0
        self.BB1_end = 0
        self.BB2_start = 0
        self.register_allocation_tab = []
        self.register_rename_tab = {}
        self.phase_3_Ins = []
        self.output_bundles = []
        
        """
        Newly provided features in PIP
        """
        self.reserve_tab = []
        self.cycle_dict = {}
        self.reset_BB0_end = 0
        self.reset_BB1_start = 0
        
        self.Allstages = []
        self.schedule_pip_tab = {}
        
        self.BB0_Bubbles = []
        self.register_allocation_pip_tab = []
        self.register_rename_pip_tab = {}
        
        self.link_consume_reg = {}
        
    # generate code blocks
    def generate_block(self):
        self.BB0 = self.Ins_set[0:self.loop_region[0]]
        self.BB1 = self.Ins_set[self.loop_region[0]:self.loop_region[1]+1]
        self.BB2 = self.Ins_set[self.loop_region[1]+1:None]
    
    # show code blocks
    def show_block(self):
        for BB0_ins in self.BB0:
            print(BB0_ins['PC'], ': ', BB0_ins['Ins'])
        print('----------')
        for BB1_ins in self.BB1:
            print(BB1_ins['PC'], ': ', BB1_ins['Ins'])
        print('----------')
        for BB2_ins in self.BB2:
            print(BB2_ins['PC'], ': ', BB2_ins['Ins'])
        
    # find Des_reg
    def find_Des_reg(self, ins):
        opcode: str = ins[0]
        assert opcode in ['add', 'addi', 'sub', 'mulu', 'ld', 'st', 'loop', 'loop.pip',\
                          'nop', 'mov'], "Undefined instruction: {}".format(i)
        if opcode in ['add', 'addi', 'sub', 'mulu', 'ld', 'mov']:
            return ins[1]
        else:
            return None
        
    # find consumed operand in each instruction
    def find_Consume_reg(self, ins):
        opcode: str = ins[0]
        assert opcode in ['add', 'addi', 'sub', 'mulu', 'ld', 'st', 'loop','loop.pip',\
                          'nop', 'mov'], "Undefined instruction: {}".format(i)
        Consume_reg = []
        if opcode in ['add', 'addi', 'sub', 'mulu']:
            Consume_reg.append(ins[2])
            if opcode != 'addi':
                Consume_reg.append(ins[3])
        elif opcode in ['ld', 'st']:
            if opcode == 'st':
                Consume_reg.append(ins[1])
            Consume_reg.append(ins[2].split('(')[1].strip(')'))
        elif opcode == 'mov':
            if ins[1].startswith('x') and ins[2].startswith('x'):
                Consume_reg.append(ins[2])
        
        # remove duplicate consume regs
        return Consume_reg
    
    # find Local Dependencies
    def find_Local_dep(self):
        BB0_ind = (0, self.loop_region[0])
        BB1_ind = (self.loop_region[0], self.loop_region[1]+1)
        BB2_ind = (self.loop_region[1]+1, self.Dependency_tab['Instr_Addr'][-1]+1)
        for (start, end) in [BB0_ind, BB1_ind, BB2_ind]:
            for i in range(start, end):
                Consume_reg = self.Dependency_tab['Consume_reg'][i]
                if bool(Consume_reg):
                    Produce_reg = self.Dependency_tab['Des_reg'][start:i]
                    Produce_ID = self.Dependency_tab['ID'][start:i]
                    Produce_Instr_Addr = self.Dependency_tab['Instr_Addr'][start:i]
                    for reg in Consume_reg:
                        if reg in Produce_reg:
                            deps = [i for i, x in enumerate(Produce_reg) if x == reg][-1]
                            self.Dependency_tab['Local_dep'][i][reg] = Produce_ID[deps]
     
    # find InterLoop Dependencies
    def find_InterLoop_dep(self):
        BB0_ind = (0, self.loop_region[0])
        BB1_ind = (self.loop_region[0], self.loop_region[1]+1)
        
        Produce_reg_BB0 = self.Dependency_tab['Des_reg'][BB0_ind[0]:BB0_ind[1]]
        Produce_ID_BB0 = self.Dependency_tab['ID'][BB0_ind[0]:BB0_ind[1]]
        Produce_Instr_Addr_BB0 = self.Dependency_tab['Instr_Addr'][BB0_ind[0]:BB0_ind[1]]
    
        for i in range(BB1_ind[0], BB1_ind[1]):
            Consume_reg = self.Dependency_tab['Consume_reg'][i]
            if bool(Consume_reg):
                Producer_reg_before = self.Dependency_tab['Des_reg'][BB1_ind[0]:i]
                Produce_reg_after = self.Dependency_tab['Des_reg'][i:BB1_ind[1]]
                Produce_ID_after = self.Dependency_tab['ID'][i:BB1_ind[1]]
                Produce_Instr_Addr_after = self.Dependency_tab['Instr_Addr'][i:None]
                
                for reg in Consume_reg:
                    if reg not in Producer_reg_before and reg in Produce_reg_after:
                        deps = [i for i, x in enumerate(Produce_reg_after) if x == reg][-1]
                        self.Dependency_tab['InterLoop_dep'][i][reg] = {'BB1':Produce_ID_after[deps]}
                        if reg in Produce_reg_BB0:
                            deps = [i for i, x in enumerate(Produce_reg_BB0) if x == reg][-1]
                            self.Dependency_tab['InterLoop_dep'][i][reg]['BB0'] = Produce_ID_BB0[deps]
                            
    
    # find Loop Invariant Dependencies
    def find_LoopInvar_dep(self):
        BB0_ind = (0, self.loop_region[0])
        BB1_ind = (self.loop_region[0], self.loop_region[1]+1)
        BB2_ind = (self.loop_region[1]+1, self.Dependency_tab['Instr_Addr'][-1]+1)
        
        Produce_reg_BB0 = self.Dependency_tab['Des_reg'][BB0_ind[0]:BB0_ind[1]]
        Produce_ID_BB0 = self.Dependency_tab['ID'][BB0_ind[0]:BB0_ind[1]]
        Produce_Instr_Addr_BB0 = self.Dependency_tab['Instr_Addr'][BB0_ind[0]:BB0_ind[1]]
        Produce_reg_BB1 = self.Dependency_tab['Des_reg'][BB1_ind[0]:BB1_ind[1]]
        Produce_reg_BB2 = self.Dependency_tab['Des_reg'][BB2_ind[0]:BB2_ind[1]]
            
        for i in range(BB1_ind[0], BB1_ind[1]):
            Consume_reg = self.Dependency_tab['Consume_reg'][i]
            if bool(Consume_reg):
                for reg in Consume_reg:
                    if reg in Produce_reg_BB0 and reg not in Produce_reg_BB1:
                        deps = [i for i, x in enumerate(Produce_reg_BB0) if x == reg][-1] 
                        self.Dependency_tab['LoopInvar_dep'][i][reg] = Produce_ID_BB0[deps]
                        
        for i in range(BB2_ind[0], BB2_ind[1]):
            Consume_reg = self.Dependency_tab['Consume_reg'][i]
            if bool(Consume_reg):
                for reg in Consume_reg:
                    if reg in Produce_reg_BB0 and reg not in Produce_reg_BB1 and reg not in Produce_reg_BB2:
                        deps = [i for i, x in enumerate(Produce_reg_BB0) if x == reg][-1]  
                        self.Dependency_tab['LoopInvar_dep'][i][reg] = Produce_ID_BB0[deps]        
        
    # find Post Loop Dependencies
    def find_PostLoop_dep(self):
        BB1_ind = (self.loop_region[0], self.loop_region[1]+1)
        BB2_ind = (self.loop_region[1]+1, self.Dependency_tab['Instr_Addr'][-1]+1)
        Produce_reg_BB1 = self.Dependency_tab['Des_reg'][BB1_ind[0]:BB1_ind[1]]
        Produce_ID_BB1 = self.Dependency_tab['ID'][BB1_ind[0]:BB1_ind[1]]
        Produce_Instr_Addr = self.Dependency_tab['Instr_Addr'][BB1_ind[0]:BB1_ind[1]]
        
        Produce_reg_BB2 = self.Dependency_tab['Des_reg'][BB2_ind[0]:BB2_ind[1]]
        
        for i in range(BB2_ind[0], BB2_ind[1]):
            Consume_reg = self.Dependency_tab['Consume_reg'][i]
            if bool(Consume_reg):
                for reg in Consume_reg:
                    if reg in Produce_reg_BB1 and reg not in Produce_reg_BB2:
                        deps = [i for i, x in enumerate(Produce_reg_BB1) if x == reg][-1]
                        self.Dependency_tab['PostLoop_dep'][i][reg] = Produce_ID_BB1[deps]
                        
    
    def dependency_analysis(self):
        All_ins = [self.BB0, self.BB1, self.BB2]
        while len(All_ins) != 0:
            BB = All_ins.pop(0)
            for ins in BB:
                self.Dependency_tab['Instr_Addr'].append(ins['PC'])
                self.Dependency_tab['ID'].append(self.ID)
                self.schedule_addr_tab[self.ID] = None
                self.Dependency_tab['opcode'].append(ins['Ins'][0])
                self.ID += 1
                self.Dependency_tab['Des_reg'].append(self.find_Des_reg(ins['Ins']))  
                self.Dependency_tab['Consume_reg'].append(self.find_Consume_reg(ins['Ins']))
                
        self.Dependency_tab['Local_dep'] = [{} for _ in range(len(self.Dependency_tab['ID']))]
        self.find_Local_dep()
        
        self.Dependency_tab['PostLoop_dep'] = [{} for _ in range(len(self.Dependency_tab['ID']))]
        self.find_PostLoop_dep()
        
        self.Dependency_tab['LoopInvar_dep'] = [{} for _ in range(len(self.Dependency_tab['ID']))]
        self.find_LoopInvar_dep()
 
        self.Dependency_tab['InterLoop_dep'] = [{} for _ in range(len(self.Dependency_tab['ID']))]
        self.find_InterLoop_dep()
        
    def show_dependency_tab(self):
        for i in range(len(self.Dependency_tab['ID'])):
            print(self.Dependency_tab['Instr_Addr'][i], self.Dependency_tab['ID'][i],\
                  self.Dependency_tab['opcode'][i], self.Dependency_tab['Des_reg'][i], \
                  self.Dependency_tab['Local_dep'][i], self.Dependency_tab['InterLoop_dep'][i],\
                  self.Dependency_tab['LoopInvar_dep'][i], self.Dependency_tab['PostLoop_dep'][i])

    
    def check_empty_slot(self, opcode, bundle: schedule_cell):
        find_slot = False 
        slot_type = None
        if opcode in ['add', 'addi', 'sub', 'mov']:
            if bundle.ALU0_empty():
                slot_type = 'ALU0'
                find_slot = True
            elif bundle.ALU1_empty():
                slot_type = 'ALU1'
                find_slot = True
        elif opcode == 'mulu':
            if bundle.Mult_empty():
                slot_type = 'Mult'
                find_slot = True
        elif opcode in ['ld', 'st']:
            if bundle.Mem_empty():
                slot_type = 'Mem'
                find_slot = True
        else:
            if bundle.Branch_empty():
                slot_type = 'Branch'
                find_slot = True
        return find_slot, slot_type
    
    def latency(self, dep_opcode):
        lat = 1
        if dep_opcode == 'mulu':
            lat = 3
        return lat
    
    def check_dependency(self, i, dep_type: str, phase=0):
        dep_result = False
        if dep_type == 'Local_dep':
            dep_result = (len(self.Dependency_tab['Local_dep'][i]) > 0)
            
        if dep_type == 'LoopInvar_dep':
            dep_result = (len(self.Dependency_tab['LoopInvar_dep'][i]) > 0)
            
        if dep_type == 'PostLoop_dep':
            dep_result = (len(self.Dependency_tab['PostLoop_dep'][i]) > 0)
            
        # To determine interloop dep
        if dep_type == 'InterLoop_dep' and phase > 0:
            if phase == 1:
                dep_key = 'BB0'
            elif phase == 2:
                dep_key = 'BB1'
            if len(self.Dependency_tab['InterLoop_dep'][i]) > 0:
                dep = self.Dependency_tab['InterLoop_dep'][i]
                for dep_reg, dep_ID in dep.items():
                    if dep_key in dep_ID:
                        dep_result = True
                        break
        if dep_type == 'InterLoop_dep' and phase == 0:
            dep_result = (len(self.Dependency_tab['InterLoop_dep'][i]) > 0)
                        
        return dep_result
    
    def get_max_desire_addr(self, i, dep_type: str):
        max_desire_addr = 0
        # with dependency
        dep = self.Dependency_tab[dep_type][i]
        # for multiple conflicts, find the max distance
        for dep_reg, dep_ID in dep.items():
            # find producer's opcode, addr in schedule, calculate distance between producer and consumer
            if dep_type == 'InterLoop_dep':
                if 'BB0' in dep_ID:
                    dep_ID = dep_ID['BB0']
                else:
                    continue
            
            dep_opcode = self.Dependency_tab['opcode'][dep_ID]
            dep_addr = self.schedule_addr_tab[dep_ID]['addr']
            desire_addr = dep_addr + self.latency(dep_opcode)
            if desire_addr > max_desire_addr:
                max_desire_addr = desire_addr
        return max_desire_addr
    
    
    def add_empty_bubble(self, num_bubble, start_bubble_addr, insert_BB0=False):
        
        # case: reduce unnecessary delay slots within the loop body
        if insert_BB0:
            self.BB0_end = self.BB0_end + num_bubble
            self.BB1_start = self.BB1_start + num_bubble
            
        # add bubbles
        for bubble_addr in range(start_bubble_addr, start_bubble_addr+num_bubble):
            self.schedule_tab.insert(bubble_addr, schedule_cell(addr=bubble_addr))
        
        # update addr of bundle in schedule_tab
        for new_addr in range(start_bubble_addr+num_bubble, len(self.schedule_tab)):
            self.schedule_tab[new_addr].set_addr(new_addr)
            
        # update schedule_addr_tab
        for bundle in self.schedule_tab:
            if not bundle.ALU0_empty():
                self.schedule_addr_tab[bundle.ALU0]['addr'] = bundle.addr
            if not bundle.ALU1_empty():
                self.schedule_addr_tab[bundle.ALU1]['addr'] = bundle.addr
            if not bundle.Mult_empty():
                self.schedule_addr_tab[bundle.Mult]['addr'] = bundle.addr
            if not bundle.Mem_empty():
                self.schedule_addr_tab[bundle.Mem]['addr'] = bundle.addr
            if not bundle.Branch_empty():
                self.schedule_addr_tab[bundle.Branch]['addr'] = bundle.addr
                
        return len(self.schedule_tab)-1
                    
    def find_schedule_slot(self, opcode, ID, addr, start_bundle_addr=0, end_bundle_addr=0, find_BB1_end=False):
        current_addr = addr
        find_slot = False
        for bundle_addr in range(start_bundle_addr, end_bundle_addr):
            find_slot, slot_type = self.check_empty_slot(opcode, self.schedule_tab[bundle_addr])
            if find_slot == True:
                # find empty slot, record and break
                self.schedule_tab[bundle_addr].set_slot(slot_type, ID)
                self.schedule_addr_tab[ID] = {'addr':bundle_addr, 'slot_type':slot_type}
                break
                
        # no empty slots, creat a new bundle, then record it      
        if find_slot == False:
            current_addr += 1
            if find_BB1_end:
                self.BB1_end += 1
                self.BB2_start += 1
            bundle = schedule_cell(addr=current_addr)
            _, slot_type = self.check_empty_slot(opcode, bundle)
            bundle.set_slot(slot_type, ID)
            self.schedule_tab.append(bundle)
            self.schedule_addr_tab[ID] = {'addr':current_addr, 'slot_type':slot_type}
            
        return current_addr
        
         
    def show_scheduling_tab(self):
        for cell in self.schedule_tab:
            cell.show_schedule_cell()
            
    def show_reserve_tab(self):
        for cell in self.reserve_tab:
            cell.show_schedule_cell()
   
    def calculate_II(self):
        num_ALU = 0
        num_Mult = 0
        num_Mem = 0
        num_Branch = 0
        for opcode in self.Dependency_tab['opcode'][self.loop_region[0]:self.loop_region[1]+1]:
            if opcode in ['add', 'addi', 'sub', 'mov']:
                num_ALU += 1
            elif opcode == 'mulu':
                num_Mult += 1
            elif opcode in ['ld', 'st']:
                num_Mem += 1
            elif opcode in ['loop', 'loop.pip']:
                num_Branch += 1
        compare = [math.ceil(num_ALU/2), num_Mult, num_Mem, num_Branch]
        return max(compare)
    
    def check_slot_type(self, opcode):
        slot_type = None
        if opcode in ['add', 'addi', 'sub', 'mov']:
            slot_type = 'ALU'
        elif opcode == 'mulu':
            slot_type = 'Mult'
        elif opcode in ['ld', 'st']:
            slot_type = 'Mem'
        else:
            slot_type = 'Branch'
        return slot_type
    
    def check_reserved_tab(self, slot_type):
        have_empty = False
        cycle_ID = 0
        cycle_ID_list = []
        
        for i in range(len(self.reserve_tab)):
            if slot_type == 'ALU':
                have_empty = self.reserve_tab[i].ALU0_empty()
                if not have_empty:
                    have_empty = self.reserve_tab[i].ALU1_empty()
            elif slot_type == 'Mult':
                have_empty = self.reserve_tab[i].Mult_empty()
            elif slot_type == 'Mem':
                have_empty = self.reserve_tab[i].Mem_empty()
            elif slot_type == 'Branch':
                have_empty = self.reserve_tab[i].Branch_empty()
                
            if have_empty:
                cycle_ID = i
                cycle_ID_list.append(cycle_ID)
        if len(cycle_ID_list) > 0:
            have_empty = True
        else:
            have_empty = False
        return have_empty, cycle_ID_list
    
    
    def reset_schedule(self, start_II):
        self.BB0_Bubbles = []
        self.Allstages = []
        self.reserve_tab = []
        addr = self.reset_BB0_end
        
        Singlestage = []
        cycle = 0
        for i in range(start_II):
            addr += 1
            Singlestage.append(schedule_cell(addr, cycle))
            self.reserve_tab.append(schedule_cell(cycle=cycle))
            cycle += 1
            
        self.Allstages.append(Singlestage)
        self.BB1_start = self.reset_BB1_start
        self.BB0_end = self.reset_BB0_end
        
        BB1_ind = (self.loop_region[0], self.loop_region[1]+1)
        self.schedule_pip_tab = {}
        for i in range(BB1_ind[0], BB1_ind[1]):
            self.schedule_pip_tab[i] = None
        

    def append_Singlestage(self, num_stages=1):
        for i in range(num_stages):
            Singlestage = copy.deepcopy(self.reserve_tab)
            addr = self.Allstages[-1][-1].addr
            for bundle in Singlestage:
                addr += 1
                bundle.set_addr(addr)
            self.Allstages.append(Singlestage)
    
    def mark_reserved_tab(self, stage, cycle, slot_type, ID):
        for i in range(len(self.Allstages)):
            if i == stage:
                continue
            else:
                slot_type = self.Allstages[i][cycle].set_reserved(slot_type, ID)  
                
    def get_max_desire_addr_pip(self, ID, dep_type: str, start_II):
        max_desire_addr = 0
        # with dependency
        dep = self.Dependency_tab[dep_type][ID]
        # for multiple conflicts, find the max stage
        for dep_reg, dep_ID in dep.items():
            # find producer's opcode, addr in schedule, calculate distance between producer and consumer
            if dep_type == 'InterLoop_dep':
                if 'BB0' in dep_ID:
                    dep_ID = dep_ID['BB0']
                else:
                    continue
            # {'stage':0, 'cycle': start_II-1}
            dep_producer = self.schedule_pip_tab[dep_ID]
            dep_opcode = self.Dependency_tab['opcode'][dep_ID]
            desire_addr = dep_producer['stage'] * start_II + dep_producer['cycle'] + self.latency(dep_opcode)
            
            if desire_addr > max_desire_addr:
                max_desire_addr = desire_addr
                
        return max_desire_addr
    
    def add_empty_bubble_pip(self, num_bubble, start_bubble_addr, insert_BB0=False):
        # case: reduce unnecessary delay slots within the loop body
        if insert_BB0:
            self.BB0_end = self.BB0_end + num_bubble
            self.BB1_start = self.BB1_start + num_bubble
            
        # add bubbles
        for bubble_addr in range(start_bubble_addr, start_bubble_addr+num_bubble):
            self.BB0_Bubbles.append(schedule_cell(addr=bubble_addr))        
                
        # update addrs in All_stages
        new_addr = start_bubble_addr+num_bubble
        for Singlestage in self.Allstages:
            for bundle in Singlestage:
                bundle.set_addr(new_addr)
                new_addr += 1
                
    def fill_reserve_tab(self, cycle, slot_type, ID, desire_stage):
        slot_type = self.reserve_tab[cycle].set_reserved(slot_type, ID)
        self.Allstages[desire_stage][cycle].set_slot(slot_type, ID)
        self.schedule_pip_tab[ID] = {'stage':desire_stage, 'cycle': cycle, 'slot_type':slot_type}
        
        self.mark_reserved_tab(stage=desire_stage, cycle=cycle, slot_type=slot_type, ID=ID)

    def fix_local_like_dep(self, desire_addr, start_II, cycle_ID_list, slot_type, ID):
        # have local dep
        desire_cycle = int(desire_addr % start_II)
        desire_stage = int((desire_addr - desire_cycle)/start_II)
        
        min_cycle_ID = min(cycle_ID_list)
        max_cycle_ID = max(cycle_ID_list)
        
        if desire_stage < len(self.Allstages):
            if desire_cycle <= max_cycle_ID:
                for cycle in cycle_ID_list:
                    if cycle >= desire_cycle:
                        self.fill_reserve_tab(cycle, slot_type, ID, desire_stage)
                        break
            else:
                next_stage = desire_stage + 1
                if next_stage >= len(self.Allstages):
                    self.append_Singlestage(num_stages=1)
                self.fill_reserve_tab(min_cycle_ID, slot_type, ID, next_stage)
                
        else:
            # case max_desire_stage >= len(self.Allstages)
            num_stages = desire_stage - len(self.Allstages) + 1
            self.append_Singlestage(num_stages=num_stages)
            if desire_cycle <= max_cycle_ID:
                for cycle in cycle_ID_list:
                    if cycle >= desire_cycle:
                        self.fill_reserve_tab(cycle, slot_type, ID, desire_stage)
                        break
            else:
                next_stage = desire_stage + 1
                if next_stage >= len(self.Allstages):
                    self.append_Singlestage(num_stages=1)
                self.fill_reserve_tab(min_cycle_ID, slot_type, ID, next_stage)
        
        
    def fix_phase2_interloop_dep_in_stage1(self, i, ID, start_II):
        reschedule_now = False
        dep = self.Dependency_tab['InterLoop_dep'][i]
        for dep_reg, dep_ID in dep.items():
            if 'BB1' in dep_ID and dep_ID['BB1'] == ID:
                producer_latency = self.latency(self.Dependency_tab['opcode'][dep_ID['BB1']])
                if producer_latency > start_II:
                    reschedule_now = True
                    break
                    
        return reschedule_now
    
    def fix_phase2_interloop_dep_in_stage2(self, i, ID, start_II):
        reschedule_now = False
        dep = self.Dependency_tab['InterLoop_dep'][i]
        consumer_stage = self.schedule_pip_tab[ID]['stage']
        consumer_cycle = self.schedule_pip_tab[ID]['cycle']
        consumer_addr = self.Allstages[consumer_stage][consumer_cycle].addr
        consumer_finish = consumer_addr + start_II
                
        for dep_reg, dep_ID in dep.items():
            if 'BB1' in dep_ID:
                producer_latency = self.latency(self.Dependency_tab['opcode'][dep_ID['BB1']])
                producer_stage = self.schedule_pip_tab[dep_ID['BB1']]['stage']
                producer_cycle = self.schedule_pip_tab[dep_ID['BB1']]['cycle']
                producer_addr = self.Allstages[producer_stage][producer_cycle].addr
                producer_latency = self.latency(self.Dependency_tab['opcode'][dep_ID['BB1']])
                producer_finish = producer_addr + producer_latency
                
                if producer_finish > consumer_finish:
                    reschedule_now = True
                    break
        return reschedule_now
        
                
    def scheduling_analysis_pip(self):

        BB0_ind = (0, self.loop_region[0])
        BB1_ind = (self.loop_region[0], self.loop_region[1]+1)
        BB2_ind = (self.loop_region[1]+1, self.Dependency_tab['Instr_Addr'][-1]+1)
        addr = 0
        bundle = schedule_cell(addr=addr)
        self.schedule_tab.append(bundle)
        
        # first schedule instructins in BB0
        for i in range(BB0_ind[0], BB0_ind[1]):
            opcode = self.Dependency_tab['opcode'][i]
            ID = self.Dependency_tab['ID'][i]
            # check Local dependency
            if self.check_dependency(i, 'Local_dep'):
                # with dependency
                max_desire_addr = self.get_max_desire_addr(i, 'Local_dep')
                if max_desire_addr > addr:
                    # add empty bubble bundles, then record it
                    num_bubble = max_desire_addr - addr
                    addr = self.add_empty_bubble(num_bubble, addr+1)
                    addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=self.schedule_tab[-1].addr, end_bundle_addr=len(self.schedule_tab))
                elif max_desire_addr == addr:
                    # find empty slot in current bundle
                    addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=self.schedule_tab[-1].addr, end_bundle_addr=len(self.schedule_tab))
                else:
                    # max_desire_addr < addr
                    addr = self.find_schedule_slot(opcode, ID, addr, end_bundle_addr=len(self.schedule_tab))
            else:
                # without dependency, then go through the table, to find a empty slot
                addr = self.find_schedule_slot(opcode, ID, addr, end_bundle_addr=len(self.schedule_tab))
                
        self.BB0_end = addr  # record final addr of BB0
        self.reset_BB0_end = self.BB0_end
        self.reset_BB1_start = self.reset_BB0_end + 1
        
        # Then schedule instructions in BB1
        
        self.BB1_start = self.BB0_end + 1
        start_II = self.calculate_II()
        schedule_finished = False
        reschedule_now = False
        self.reset_schedule(start_II)
        
        while not schedule_finished:
            for i in range(BB1_ind[0], BB1_ind[1]):
                opcode = self.Dependency_tab['opcode'][i]
                ID = self.Dependency_tab['ID'][i]
                slot_type = self.check_slot_type(opcode) # return ALU
                
                # check empty slot in reserve_tab
                have_empty, cycle_ID_list = self.check_reserved_tab(slot_type)
                
                if not have_empty:
                    # all slots are reserved, rescheduling
                    reschedule_now = True
                    break
                else:
                    # find empty slots
                    
                    # check dep
                    if not (self.check_dependency(i, 'Local_dep') or self.check_dependency(i, 'LoopInvar_dep') or self.check_dependency(i, 'InterLoop_dep', phase=1)):
                        # without dependency
                        if opcode == 'loop' or opcode == 'loop.pip':
                            # (1) loop instruction: (a) set reserve_tab; (b) fill ID into the slot of stage 0; (c) store position; (d) set reserved flag in ALLstages
                            self.fill_reserve_tab(start_II-1, 'Branch', ID, 0)
                        else:
                            # (2) normal instruction: only have phase 2, without all deps
                            if self.check_dependency(i, 'InterLoop_dep', phase=2):
                                # only have phase 2 interloop
                                reschedule_now, start_II, addr = self.fix_phase2_interloop_dep_in_stage1(i, ID, start_II)
                                if reschedule_now:
                                    break
                            else:
                                # without all deps or meet the phase 2 deps
                                # (a): find earlist slot in stage 0, (b) set reserve_tab; (c) fill ID into the slot of stage 0; (d) store position; (e) set reserved flag in ALLstages
                                self.fill_reserve_tab(cycle_ID_list[0], slot_type, ID, 0)
                    else:
                        # might interloop (phase-1), loop-inv
                        loop_desire_addr = None
                        if self.check_dependency(i, 'LoopInvar_dep'):
                            loop_desire_addr = self.get_max_desire_addr(i, 'LoopInvar_dep')
                        if self.check_dependency(i, 'InterLoop_dep', phase=1):
                            if not loop_desire_addr:
                                loop_desire_addr = self.get_max_desire_addr(i, 'InterLoop_dep')
                            else:
                                loop_desire_addr = max(loop_desire_addr, self.get_max_desire_addr(i, 'InterLoop_dep'))
                        
                        # might have local
                        if self.check_dependency(i, 'Local_dep'):
                            # have local dep
                            max_desire_addr = self.get_max_desire_addr_pip(ID, 'Local_dep', start_II)
                            self.fix_local_like_dep(max_desire_addr, start_II, cycle_ID_list, slot_type, ID)

                            # integrate all deps
                            real_bundle = self.schedule_pip_tab[ID]
                            real_addr = self.Allstages[real_bundle['stage']][real_bundle['cycle']].addr
                            if loop_desire_addr:
                                if loop_desire_addr > real_addr:
                                    # add bubbles in BB0
                                    num_bubble = loop_desire_addr - real_addr
                                    self.add_empty_bubble_pip(num_bubble, self.BB1_start, insert_BB0=True)
                        else:
                            # without local dependency
                            self.fill_reserve_tab(min(cycle_ID_list), slot_type, ID, 0)
                            real_bundle = self.schedule_pip_tab[ID]
                            real_addr = self.Allstages[real_bundle['stage']][real_bundle['cycle']].addr
                            if loop_desire_addr > real_addr:
                                num_bubble = loop_desire_addr - real_addr
                                self.add_empty_bubble_pip(num_bubble, self.BB1_start, insert_BB0=True)
                                
                        # solve above conflicts, now check phase-2 dep
                        if self.check_dependency(i, 'InterLoop_dep', phase=2):
                            # only have phase 2 interloop
                            reschedule_now = self.fix_phase2_interloop_dep_in_stage1(i, ID, start_II)
                            if reschedule_now:
                                break
                                    
            
            if reschedule_now:
                start_II += 1
                self.reset_schedule(start_II)
                reschedule_now = False
            else:
                # phase 1 finished, now phase 2, check all interloop
                for i in range(BB1_ind[0], BB1_ind[1]):
                    if self.check_dependency(i, 'InterLoop_dep', phase=2):
                        reschedule_now = self.fix_phase2_interloop_dep_in_stage2(i, ID, start_II)
                        if reschedule_now:
                            start_II += 1
                            self.reset_schedule(start_II)
                            reschedule_now = False
                            break
                schedule_finished = True
                
        # append bubbles in BB0 and Allstages into 
        for bubble in self.BB0_Bubbles:
            self.schedule_tab.append(bubble)
        for Singlestage in self.Allstages:
            for cell in Singlestage:
                self.schedule_tab.append(cell)
            
        self.BB1_end = self.Allstages[-1][-1].addr  # record final addr of BB1
        
        
        # update self.schedule_addr_tab
        for ID in range(BB1_ind[0], BB1_ind[1]):
            consumer_stage = self.schedule_pip_tab[ID]['stage']
            consumer_cycle = self.schedule_pip_tab[ID]['cycle']
            consumer_slot_type = self.schedule_pip_tab[ID]['slot_type']
            consumer_addr = self.Allstages[consumer_stage][consumer_cycle].addr
            self.schedule_addr_tab[ID] = {'addr':consumer_addr, 'slot_type':consumer_slot_type}
            

        
        # Then schedule instructins in BB2
        # a new portion for BB2
        addr = self.BB1_end + 1
        self.BB2_start = addr
        self.schedule_tab.append(schedule_cell(addr))
        for i in range(BB2_ind[0], BB2_ind[1]):
            opcode = self.Dependency_tab['opcode'][i]
            ID = self.Dependency_tab['ID'][i]
            
            if not (self.check_dependency(i, 'Local_dep') or self.check_dependency(i, 'LoopInvar_dep') or self.check_dependency(i, 'PostLoop_dep')):
                # without dependency
                addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=self.BB2_start, end_bundle_addr=len(self.schedule_tab))
            else:
    
                # first compare PostLoop dep and LoopInvar dep
                loop_desire_addr = None
                if self.check_dependency(i, 'LoopInvar_dep'):
                    loop_desire_addr = self.get_max_desire_addr(i, 'LoopInvar_dep')
                if self.check_dependency(i, 'PostLoop_dep'):
                    if not loop_desire_addr:
                        loop_desire_addr = self.get_max_desire_addr(i, 'PostLoop_dep')
                    else:
                        loop_desire_addr = max(loop_desire_addr, self.get_max_desire_addr(i, 'PostLoop_dep'))
                        
                # then check local dependency 
                if self.check_dependency(i, 'Local_dep'):
                    max_desire_addr = self.get_max_desire_addr(i, 'Local_dep')
                    if max_desire_addr > addr:
                        # add empty bubble bundles, then record it
                        num_bubble = max_desire_addr - addr
                        addr = self.add_empty_bubble(num_bubble, addr+1)
                        start_addr = addr
                    elif max_desire_addr == addr:
                        start_addr = addr
                    else:
                        # max_desire_addr < addr
                        start_addr = max_desire_addr
                        
                    # integrate all deps
                    if loop_desire_addr:
                        if loop_desire_addr <= start_addr:
                            addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=start_addr, end_bundle_addr=len(self.schedule_tab))
                        else:
                            # loop_desire_addr > start_addr
                            # add bubbles in BB2
                            num_bubble = loop_desire_addr - start_addr
                            addr = self.add_empty_bubble(num_bubble, self.BB2_start)
                            start_addr = start_addr + num_bubble
                            addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=start_addr, end_bundle_addr=len(self.schedule_tab))
                    else:
                        # only Local dep
                        addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=start_addr, end_bundle_addr=len(self.schedule_tab))
                else:
                    # without local dependency 
                    if loop_desire_addr <= self.BB2_start:
                        addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=self.BB2_start, end_bundle_addr=len(self.schedule_tab))
                    else:
                        # loop_desire_addr > self.BB1_start:
                        if loop_desire_addr <= addr:
                            # loop_desire_addr <= addr:
                            addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=loop_desire_addr, end_bundle_addr=len(self.schedule_tab))
                        else:
                            # loop_desire_addr > addr:
                            num_bubble = loop_desire_addr - addr
                            addr = self.add_empty_bubble(num_bubble, self.BB2_start)
                            addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=self.schedule_tab[-1].addr, end_bundle_addr=len(self.schedule_tab))
        

    def check_imm(self, opcode):
        with_imm = False
        imm_ind = None
        if opcode in ['ld', 'st']:
            with_imm = True
            imm_ind = 2
        if opcode == 'addi':
            imm_ind = 3
            with_imm = True
        return with_imm, imm_ind   
    
    def register_allocating_pip(self):
        # initialize the self.register_rename_tab:
        for ID in self.Dependency_tab['ID']:
            self.register_rename_tab[ID] = None
            self.link_consume_reg[ID] = {}
        
        # phase 1: allocating registers to BB1 instructions
        rotate_reg_ID = 32
        incre_RID = len(self.Allstages) + 1
        for i in range(self.BB1_start, self.BB1_end+1):
            for slot_type in ['ALU0', 'ALU1', 'Mult', 'Mem', 'Branch']:
                if self.schedule_tab[i].return_ID(slot_type) not in ['nop', 'Reserved']:
                    ID = self.schedule_tab[i].return_ID(slot_type)
                    opcode = self.Dependency_tab['opcode'][ID]
                    Des_reg = self.Dependency_tab['Des_reg'][ID]
                    if Des_reg:
                        self.register_rename_tab[ID] = rotate_reg_ID
                        if bool(self.Dependency_tab['Consume_reg'][ID]):
                            # with dep
                            self.schedule_tab[i].set_slot(slot_type, {ID: [opcode, 'x'+str(rotate_reg_ID)]})
                        else:
                            # without dep
                            Ins = self.Ins_set[ID]['Ins']
                            Ins[1] = 'x'+str(rotate_reg_ID)
                            self.schedule_tab[i].set_slot(slot_type, {ID: Ins})
                        rotate_reg_ID += incre_RID
                    else:
                        # opcode = loop, st, (nop)
                        if opcode == 'loop.pip':
                            self.schedule_tab[i].set_slot(slot_type, {ID: self.Ins_set[ID]['Ins']})
                        if opcode == 'st':
                            self.schedule_tab[i].set_slot(slot_type, {ID: [opcode]})
                            
                            
        # phase 2: solve loop invariant register
        BB1_ind = (self.loop_region[0], self.loop_region[1]+1)
        Produce_reg_BB1 = self.Dependency_tab['Des_reg'][BB1_ind[0]:BB1_ind[1]]
        reg_ID = 1
        for i in range(0, self.BB1_start):
            for slot_type in ['ALU0', 'ALU1', 'Mult', 'Mem', 'Branch']:
                
                if self.schedule_tab[i].return_ID(slot_type) not in ['nop', 'Reserved']:
                    ID = self.schedule_tab[i].return_ID(slot_type)
                    opcode = self.Dependency_tab['opcode'][ID]
                    Des_reg = self.Dependency_tab['Des_reg'][ID]
                    if Des_reg not in ['LC', 'EC', None] and Des_reg not in Produce_reg_BB1:
                        self.register_rename_tab[ID] = reg_ID
                        reg_ID += 1
                    elif Des_reg in ['LC', 'EC']:
                        self.schedule_tab[i].set_slot(slot_type, {ID: self.Ins_set[ID]['Ins']})
                    else:
                        if opcode == 'st':
                            self.schedule_tab[i].set_slot(slot_type, {ID: [opcode]})
        
        # phase 3: link consumer in BB1 to producer
        BB0_ind = (0, self.loop_region[0])
        Produce_reg_BB0 = self.Dependency_tab['Des_reg'][BB0_ind[0]:BB0_ind[1]]
        
        for i in range(self.loop_region[0], self.loop_region[1]+1):
            if self.Dependency_tab['Consume_reg'][i]:
                link_reg = {}
                ID = self.Dependency_tab['ID'][i]
                for dep_type in ['Local_dep', 'InterLoop_dep', 'LoopInvar_dep', 'PostLoop_dep']:
                    if self.check_dependency(i, dep_type):
                        # with dep
                        dep = self.Dependency_tab[dep_type][i]
                        for dep_reg, dep_ID in dep.items():
                            if dep_type == 'Local_dep':
                                # {4: {'stage': 0, 'cycle': 0, 'slot_type': 'Mem'}
                                stage_offset = self.schedule_pip_tab[ID]['stage'] - self.schedule_pip_tab[dep_ID]['stage']
                                link_reg[dep_reg] = 'x'+str(self.register_rename_tab[dep_ID] + stage_offset)
                                
                            if dep_type == 'InterLoop_dep':
                                dep_BB1_ID = dep_ID['BB1']
                                stage_offset = self.schedule_pip_tab[ID]['stage'] - self.schedule_pip_tab[dep_BB1_ID]['stage']
                                link_reg[dep_reg] = 'x'+str(self.register_rename_tab[dep_BB1_ID] + stage_offset + 1)
                                if 'BB0' in dep_ID:
                                    dep_BB0_ID = [BB0_ID for (BB0_ID, value) in enumerate(Produce_reg_BB0) if value == dep_reg]
                                    for singleID in dep_BB0_ID:
                                        self.register_rename_tab[singleID] = self.register_rename_tab[dep_BB1_ID] + 1 - self.schedule_pip_tab[dep_BB1_ID]['stage']
                                    
                            if dep_type == 'LoopInvar_dep':
                                link_reg[dep_reg] = 'x'+str(self.register_rename_tab[dep_ID])
                            
                addr = self.schedule_addr_tab[i]['addr']
                slot_type = self.schedule_addr_tab[i]['slot_type']
                
                for consume_reg in self.Dependency_tab['Consume_reg'][i]:
                    self.schedule_tab[addr].append_consume_reg_slot(i, slot_type, link_reg[consume_reg])
                    
                opcode = self.Dependency_tab['opcode'][i]
                with_imm, imm_ind = self.check_imm(opcode)
                if with_imm:
                    imm = self.Ins_set[self.Dependency_tab['Instr_Addr'][i]]['Ins'][imm_ind].split('(')[0]
                    self.schedule_tab[addr].insert_imm(i, slot_type, imm_ind, imm)
                    
        
        # phase 4: fix remaining register
        
        # BB0: allocate remaining dest register in BB0
        for ID in range(0, self.loop_region[0]):
            rename_Des_reg = self.register_rename_tab[ID]
            Des_reg = self.Dependency_tab['Des_reg'][ID]
            if Des_reg not in ['LC', 'EC', 'None'] and rename_Des_reg == None:
                self.register_rename_tab[ID] = reg_ID
                reg_ID += 1
                    
        # BB2: allocate remaining dest register in BB2
        for ID in range(self.loop_region[1]+1, len(self.Dependency_tab['ID'])):
            rename_Des_reg = self.register_rename_tab[ID]
            Des_reg = self.Dependency_tab['Des_reg'][ID]
            if Des_reg != None and rename_Des_reg == None:
                self.register_rename_tab[ID] = reg_ID
                reg_ID += 1
                    
                   
        # link consumers in BB0
        for ID in range(0, self.loop_region[0]):
            opcode = self.Dependency_tab['opcode'][ID]
            Des_reg = self.Dependency_tab['Des_reg'][ID]
            slot_type = self.schedule_addr_tab[ID]['slot_type']
            addr = self.schedule_addr_tab[ID]['addr']
            if self.Dependency_tab['Consume_reg'][ID]:
                link_reg = {}
                dep = self.Dependency_tab['Local_dep'][ID]
                for dep_reg, dep_ID in dep.items():
                    link_reg[dep_reg] = 'x'+str(self.register_rename_tab[dep_ID]) 
                    
                if opcode != 'st':
                    Ins = [opcode, 'x'+str(self.register_rename_tab[ID])]
                    self.schedule_tab[addr].set_slot(slot_type, {ID: Ins})
                    
                for consume_reg in self.Dependency_tab['Consume_reg'][ID]:
                    self.schedule_tab[addr].append_consume_reg_slot(ID, slot_type, link_reg[consume_reg])
                    
                with_imm, imm_ind = self.check_imm(opcode)
                if with_imm:
                    imm = self.Ins_set[self.Dependency_tab['Instr_Addr'][ID]]['Ins'][imm_ind].split('(')[0]
                    
                    self.schedule_tab[addr].insert_imm(ID, slot_type, imm_ind, imm)
                    
            elif Des_reg not in ['LC', 'EC']:
                Ins = self.Ins_set[ID]['Ins']
                Ins[1] = 'x'+str(self.register_rename_tab[ID])
                self.schedule_tab[addr].set_slot(slot_type, {ID: Ins})
                
        
        
        # link consumers in BB2
        for ID in range(self.loop_region[1]+1, self.Dependency_tab['Instr_Addr'][-1]+1):
            opcode = self.Dependency_tab['opcode'][ID]
            Des_reg = self.Dependency_tab['Des_reg'][ID]
            slot_type = self.schedule_addr_tab[ID]['slot_type']
            addr = self.schedule_addr_tab[ID]['addr']
            if self.Dependency_tab['Consume_reg'][ID]:
                link_reg = {}
                for dep_type in ['Local_dep', 'LoopInvar_dep', 'PostLoop_dep']:
                    if self.check_dependency(ID, dep_type):
                        dep = self.Dependency_tab[dep_type][ID]
                        for dep_reg, dep_ID in dep.items():
                            if dep_type == 'PostLoop_dep':
                                link_reg[dep_reg] = 'x'+str(self.register_rename_tab[dep_ID] + len(self.Allstages)-1 - self.schedule_pip_tab[dep_ID]['stage']) 
                            else:
                                link_reg[dep_reg] = 'x'+str(self.register_rename_tab[dep_ID])
                                
                if opcode == 'st':
                    Ins = [opcode]
                else:
                    Ins = [opcode, 'x'+str(self.register_rename_tab[ID])]
                
                self.schedule_tab[addr].set_slot(slot_type, {ID: Ins})
                for consume_reg in self.Dependency_tab['Consume_reg'][ID]:
                    self.schedule_tab[addr].append_consume_reg_slot(ID, slot_type, link_reg[consume_reg])
                    
                with_imm, imm_ind = self.check_imm(opcode)
                if with_imm:
                    imm = self.Ins_set[self.Dependency_tab['Instr_Addr'][ID]]['Ins'][imm_ind].split('(')[0]
                    self.schedule_tab[addr].insert_imm(ID, slot_type, imm_ind, imm)                                
            else:
                Ins = self.Ins_set[ID]['Ins']
                Ins[1] = 'x'+str(self.register_rename_tab[ID])
                self.schedule_tab[addr].set_slot(slot_type, {ID: Ins})
                
        
        # prepare the loop
        
        # compact BB1
        num_cycles = len(self.reserve_tab)
        cycle = 0
        BB1_compact = []
        for i in range(num_cycles):
            BB1_compact.append(schedule_cell(cycle=cycle))
            cycle += 1
        for ID, schedule_info in self.schedule_pip_tab.items():
            ins_addr = self.schedule_addr_tab[ID]['addr']
            ins_slot_type = self.schedule_addr_tab[ID]['slot_type']
            ins_info = self.schedule_tab[ins_addr].return_ID(ins_slot_type)
            ins_cycle = schedule_info['cycle']
            BB1_compact[ins_cycle].set_slot(ins_slot_type, ins_info)
        
        # copy BB2
        BB2_copy = copy.deepcopy(self.schedule_tab[self.BB2_start:None])
        
        # schedule mov in BB0
        # ASAP for mov
        EC = len(self.Allstages) - 1
        prepare = [{-1:['mov', 'p32', 'true']}, {-2:['mov', 'EC', str(EC)]}]
        for i in range(0, self.BB1_start):
            if self.schedule_tab[i].ALU0_empty():
                self.schedule_tab[i].set_slot('ALU0', prepare.pop(0))
                if len(prepare) == 0:
                    break
            if self.schedule_tab[i].ALU1_empty():
                self.schedule_tab[i].set_slot('ALU1', prepare.pop(0))
                if len(prepare) == 0:
                    break
        if len(prepare) > 0:
            self.schedule_tab.insert(self.BB1_start, schedule_cell(addr=self.BB1_start))
            self.schedule_tab[self.BB1_start].set_slot('ALU0', prepare.pop(0))
            if len(prepare) > 0:
                self.schedule_tab[self.BB1_start].set_slot('ALU1', prepare.pop(0))
            self.BB1_start += 1        
        BB0_copy = copy.deepcopy(self.schedule_tab[0:self.BB1_start])
        
        
        
        self.schedule_tab = []
        for block in [BB0_copy, BB1_compact, BB2_copy]:
            for bundle in block:
                self.schedule_tab.append(bundle)
                
        # only for check
        for addr, bundle in enumerate(self.schedule_tab):
            bundle.set_addr(addr)
            if bundle.ALU0 == 'Reserved':
                bundle.ALU0 = 'nop'
            if bundle.ALU1 == 'Reserved':
                bundle.ALU1 = 'nop'
            if bundle.Mult == 'Reserved':
                bundle.Mult = 'nop'
            if bundle.Branch == 'Reserved':
                bundle.Branch = 'nop'
            if bundle.Mem == 'Reserved':
                bundle.Mem = 'nop'       
        
    def output_results(self):
        for bundle in self.schedule_tab:
            single_bundle = []
            for slot_type in ['ALU0', 'ALU1', 'Mult', 'Mem', 'Branch']:
                ID, cell = bundle.output_schedule_cell(slot_type)
                if ID in self.schedule_pip_tab:
                    if slot_type == 'Branch':
                        cell = cell.split(' ')[0]+' '+str(self.BB1_start)
                    else:
                        predicate = '(p'+str(self.schedule_pip_tab[ID]['stage']+32)+')'
                        cell = predicate +' '+ cell
                single_bundle.append(cell)
            self.output_bundles.append(single_bundle)
        

        
    def readInstructions(self, all_instructions):
        PC = 0
        while len(all_instructions) != 0:
            instruction = all_instructions.pop(0).replace(',', '').split(' ')
            DIR_cell = {'PC':PC, 'Ins':instruction}
            if instruction[0] == 'loop' or instruction[0] == 'loop.pip':
                self.loop_region = (int(instruction[1]), PC)
            self.Ins_set.append(DIR_cell)
            PC += 1

In [4]:
def parseInstructions(file_name='handout_pip.json'):
    json_file = open(file_name)
    all_instructions = json.load(json_file)    
    return all_instructions

In [5]:
def saveOutput(file_name='loop_result_pip.json'):
    with open(file_name, 'w') as fp:
        fp.write(
            '[\n\t' + ',\n\t'.join(json.dumps(bundle) for bundle in compiler.output_bundles) + '\n]')

#### Run the Compiler for loop.pip

In [6]:
compiler = Compiler_state()
all_instructions = parseInstructions(file_name='handout_pip.json')
compiler.readInstructions(all_instructions)

compiler.generate_block()
compiler.dependency_analysis()
compiler.scheduling_analysis_pip()
compiler.register_allocating_pip()
compiler.output_results()
saveOutput(file_name='loop_pip_result.json')